In [0]:

from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

In [0]:
# weights for pytorch model
!wget http://www.robots.ox.ac.uk/~albanie/models/pytorch-mcn/vgg_face_dag.pth
!wget --user adiencedb --password adience http://www.cslab.openu.ac.il/download/adiencedb/AdienceBenchmarkOfUnfilteredFacesForGenderAndAgeClassification/faces.tar.gz

In [0]:
! mkdir -p /content/data
! mkdir -p /content/weight
! mkdir -p /content/source


In [0]:
! cp vgg_face_dag.pth weight/
!rm -r vgg_face_dag.pth

In [0]:
!cp /content/gdrive/'My Drive'/source.tar.gz /content/source

In [0]:
! tar -xf source/source.tar.gz -C .
! rm -r source/source.tar.gz

In [0]:
! tar -xf faces.tar.gz -C data/
! rm faces.tar.gz

In [0]:
import pandas as pd

In [0]:
!wget --user adiencedb --password adience http://www.cslab.openu.ac.il/download/adiencedb/AdienceBenchmarkOfUnfilteredFacesForGenderAndAgeClassification/fold_0_data.txt
!wget --user adiencedb --password adience http://www.cslab.openu.ac.il/download/adiencedb/AdienceBenchmarkOfUnfilteredFacesForGenderAndAgeClassification/fold_1_data.txt
!wget --user adiencedb --password adience http://www.cslab.openu.ac.il/download/adiencedb/AdienceBenchmarkOfUnfilteredFacesForGenderAndAgeClassification/fold_2_data.txt
!wget --user adiencedb --password adience http://www.cslab.openu.ac.il/download/adiencedb/AdienceBenchmarkOfUnfilteredFacesForGenderAndAgeClassification/fold_3_data.txt
!wget --user adiencedb --password adience http://www.cslab.openu.ac.il/download/adiencedb/AdienceBenchmarkOfUnfilteredFacesForGenderAndAgeClassification/fold_4_data.txt

In [0]:
!cp *.txt data/
!rm -r *.txt

In [0]:
frames = []
folder_path = "data/"

for i in range(5):
    temp_df = pd.read_csv(folder_path + "fold_" + str(i) + "_data.txt", delimiter="\t")
    frames.append(temp_df)

df = pd.concat(frames)

In [0]:
df.head()

In [0]:
"""
FINAL CLASSES
(00, 02)
(04, 06)
(08, 12)
(15, 20)
(25, 32)
(38, 43)
(48, 53)
(60, 100)
None
"""

# for cleaning anomalies
map_dict = {
    "13": "(08, 12)",
    "2" : "(00, 02)",
    "22": "(15, 20)",
    "23": "(25, 32)",
    "29": "(25, 32)",
    "3" : "(00, 02)",
    "32": "(25, 32)",
    "34": "(25, 32)",
    "35": "(25, 32)",
    "36": "(38, 43)",
    "42": "(38, 43)",
    "45": "(38, 43)",
    "46": "(48, 53)",
    "55": "(48, 53)",
    "56": "(48, 53)",
    "57": "(60, 100)",
    "58": "(60, 100)",
    
    "(8, 23)" : "(08, 12)",
    "(27, 32)": "(25, 32)",
    "(38, 42)": "(38, 43)",
    "(38, 48)": "(38, 43)",

    "(00, 02)" : "(00, 02)",
    "(04, 06)" : "(04, 06)",
    "(08, 12)" : "(08, 12)",
    "(15, 20)" : "(15, 20)",
    "(25, 32)" : "(25, 32)",
    "(38, 43)" : "(38, 43)",
    "(48, 53)" : "(48, 53)",
    "(60, 100)": "(60, 100)"
}

def map_func(x):
    if x in map_dict:
        return map_dict[x]
    else:
        return x

df["age"] = df["age"].map(map_func)

In [0]:
df["age"].value_counts()

In [0]:
import os
import shutil

In [0]:
folder_path = "/content/data/faces/"

In [0]:
file_list = []
formats = ["jpg", "png"]

for subdir in os.listdir(folder_path):
    subpath = os.path.join(folder_path, subdir)
    if os.path.isdir(subpath):
        for f in os.listdir(subpath):
            filepath = os.path.join(subpath, f)
            part = f.split(".")
            if os.path.isfile(filepath) and part[-1] in formats:
                file_list.append((filepath, f))

print(len(file_list))

In [0]:
# DONT INTERRUPT WHILE RUNNING
# SHOWS PROGRESS

im_len = len(file_list)

for i, (filepath, filename) in enumerate(file_list):

    # get the identifiers
    parts = filename.split(".")
    user_id = filepath.split("/")[-2]
    file_id = parts[-2] + "." + parts[-1]
    face_id = int(parts[-3])

    # find class
    class_ = df[
       (df["user_id"] == user_id) & 
       (df["original_image"] == file_id) & 
       (df["face_id"] == face_id)
    ]["age"].values[0]

    new_path = os.path.join(folder_path, class_)
    
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    # move file
    new_path = os.path.join(new_path, filename)
    shutil.move(filepath, new_path)
    
    # progress
    prog = (20 * (i + 1)) // im_len
    print("\r[" + "="*prog + "_"*(20-prog) + "]", end="")

In [0]:
# delete unwanted files or empty folders
folder_path = "/content/data/faces/"

for subdir in os.listdir(folder_path):
    subpath = os.path.join(folder_path, subdir)

    if os.path.isdir(subpath):
        if subdir[0] != "(":
            shutil.rmtree(subpath)
    elif os.path.isfile(subpath):
        os.remove(subdir)

In [0]:
# list the classes
! ls data/faces

In [0]:
! mkdir -p 'data/faces/(00, 02)'
! mkdir -p 'data/faces/(04, 06)'
! mkdir -p 'data/faces/(08, 12)'

In [0]:
! mv 'data/faces/(0, 2)'/* 'data/faces/(00, 02)'
! mv 'data/faces/(4, 6)'/* 'data/faces/(04, 06)'
! mv 'data/faces/(8, 12)'/* 'data/faces/(08, 12)'

In [0]:
! rm -rf 'data/faces/(0, 2)'
! rm -rf 'data/faces/(4, 6)'
! rm -rf 'data/faces/(8, 12)'

In [0]:
! ls data/faces

In [0]:
import torch
import source.models as models
import source.worker as worker
import source.loader as loader

In [0]:
# the mean and std of dataset are found by running this
# takes some time to iterate twice
loader.find_mean_std("data/faces")

In [0]:
loader.random_scale = (0.8, 1.0)
loader.mean = [0.437, 0.340, 0.304]
loader.std  = [0.286, 0.252, 0.236]

In [0]:
# Dataset Loader to feed into network
# 20% of data is used for validation
loaders = loader.split_loader("data/faces", valid_frac=0.2, batch_size=32)

In [0]:
# pretrained weights - for convolution layers
state = loader.load_pth("weight/vgg_face_dag.pth")

In [0]:
model = models.vgg16(num_classes=8)

In [0]:
# pretrained vgg-face
model.load_weights(state)

In [0]:
# gives parameter count and memory in MB
model.memory_usage()

In [0]:
# train for one epoch
worker.train(model, loaders, lr=0.01, epochs=15)

In [0]:
check = loader.load_pth("checkpoint.pth")
model.load_weights(check["state_dict"])

In [0]:
valid_loader = loaders[1]
conf_mat = worker.confusion_matrix(model, valid_loader)

In [0]:
for row in conf_mat:
    for elem in row:
        print("%.2f"%(elem*100), end="\t")
    print("")

In [0]:
# one-off accuracy
ncls = len(conf_mat)
tot_acc = 0

for i in range(ncls):
    
    acc = conf_mat[i][i]

    # add left
    if i > 0:
        acc += conf_mat[i][i-1]
    
    if i < ncls - 1:
        acc += conf_mat[i][i+1]
    
    tot_acc += acc

tot_acc = tot_acc / ncls
print("%.2f" % (tot_acc * 100))

In [0]:
import os
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

In [0]:
# get index to class mapping
dire = "data/faces"
classes = os.listdir(dire)
classes.sort()
idx_to_class = {i:classes[i] for i in range(len(classes))}
print(idx_to_class)

In [0]:
# turn model to evaluation and move to cpu
model.eval()
batch_size = 32
model.to(torch.device("cpu"))

In [0]:

# inverse transform to show images
mean = loader.mean
std  = loader.std
mn_inv = [-m/s for m, s in zip(mean, std)]
sd_inv = [1/s for s in std]
inv_transform = transforms.Normalize(mean=mn_inv, std=sd_inv)

In [0]:
with torch.no_grad():
    for i, (input, target) in enumerate(valid_loader):

        output = model(input)
        _, preds = torch.max(output, 1)

        fig=plt.figure(figsize=(15, 15))
        columns = 4
        rows = 5

        for i in range(1, columns*rows + 1):

            pred_class = idx_to_class[int(preds[i])]
            real_class = idx_to_class[int(target[i])]

            ax = fig.add_subplot(rows, columns, i)
            ax.title.set_text("pred:" + pred_class + "," + "real:" + real_class)
            ax.axis("off")

            plt.imshow(inv_transform(input[i]).permute(1, 2, 0))

        break

plt.show()

In [0]:
del loaders
del model
del state
torch.cuda.empty_cache()

In [0]:
import importlib
importlib.reload(models)
importlib.reload(worker)
importlib.reload(loader)